In [1]:
import os
os.chdir("..")

In [6]:
import django
# In case that we need it later
#from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'DevEstate.settings')
# This is for async, in case we are going to see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db import transaction
from django.db.models import Q
from django.utils.dateparse import parse_datetime
from listings.models import Property, PropertyFeatures, Agent, PropertyAgent, School, PropertySchool, PriceHistory, NearbyHomes
# WORKING API CALL

#Import Necessities: SSL is a Temporary fix for a handshake bug, will fix later
import http.client
import ssl
import time
import json
import api_secrets
import propertylist
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Returns a string of JSON Information regarding every property in New York City
def fetch_all_data():
   conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
   headers = {
        'X-RapidAPI-Key': api_secrets.key,
        'X-RapidAPI-Host': api_secrets.host
   }
   conn.request("GET", "/searchByUrl?url=https://www.zillow.com/new-york-ny/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-74.15820883203124%2C%22east%22%3A-73.66382406640624%2C%22south%22%3A40.57279370530749%2C%22north%22%3A40.93585324525284%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A6181%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D", headers=headers)
   res = conn.getresponse()
   data = res.read()
   return(data.decode("utf-8"))

#Converting that string into a Dictionary
listingsInfo = json.loads(fetch_all_data()) #ALL data is now in a dictionary

#Returns a variety of information on a property based on its unique ZPID (Zillow ID)
def fetch_individual_data(ZPID):
    conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
    headers = {
        'X-RapidAPI-Key': api_secrets.key,
        'X-RapidAPI-Host': api_secrets.host
    }
    conn.request("GET", "/property?zpid=" + str(ZPID), headers=headers)
    res = conn.getresponse()
    data = res.read()
    conn.close()  # Close the connection after the request
    return data.decode("utf-8")

#Parses through a Dictionary of Listings in order to obtain each one's ZPID; Then, fetches the data associated with each one
def parseListings(listingDataDictionary):
    listingDataDictionaryList = []
    props = listingDataDictionary.get("props", [])
    
    for i, prop in enumerate(props):
        try:
            ZPID = prop.get("zpid")
            if ZPID:  # Only proceed if ZPID is present
                print(f"Fetching data for ZPID: {ZPID}")
                individual_property_data = fetch_individual_data(ZPID)
                listingDataDictionaryList.append(json.loads(individual_property_data))
                time.sleep(1)  # Sleep for 1 second between API calls to avoid rate limiting
            else:
                print(f"No ZPID found for property at index {i}")
        except Exception as e:
            print(f"Error fetching data for property at index {i}: {e}")
            # Optionally, break or continue depending on whether you want to stop at the first error or keep going
            # break

    return listingDataDictionaryList
#print(parseListings(listingsInfo))


In [7]:
def parse_datetime_safe(datetime_string):
    try:
        return parse_datetime(datetime_string)
    except (ValueError, TypeError):
        return None


def save_properties_and_related_data(property_list):
    with transaction.atomic():
        for property_data in property_list:
            # Create or update the Property
            property_defaults = {
                'street_address': property_data['address']['streetAddress'],
                'city': property_data['address']['city'],
                'state': property_data['address']['state'],
                'zipcode': property_data['address']['zipcode'],
                'country': property_data['country'],
                'latitude': property_data['latitude'],
                'longitude': property_data['longitude'],
                'living_area': property_data['livingAreaValue'],
                'bedrooms': property_data['bedrooms'],
                'bathrooms': property_data['bathrooms'],
                'year_built': property_data['yearBuilt'],
                'price': property_data['price'],
                'date_posted': parse_datetime_safe(property_data['datePosted']),
                'date_sold': parse_datetime_safe(property_data.get('dateSold')),
                'home_type': property_data['homeType'],
                'property_tax_rate': property_data['propertyTaxRate'],
                'time_on_zillow': property_data['timeOnZillow'],
                'home_status': property_data['homeStatus'],
                'annual_homeowners_insurance': property_data['annualHomeownersInsurance'],
                'rent_zestimate': property_data['rentZestimate'],
                'brokerage_name': property_data['brokerageName'],
                'page_view_count': property_data['pageViewCount'],
                'description': property_data['description'],
            }
            property_obj, created = Property.objects.update_or_create(
                zpid=property_data['zpid'], defaults=property_defaults
            )

            # Handle PropertyFeatures
            features_data = property_data.get('resoFacts', {})
            PropertyFeatures.objects.update_or_create(
                property=property_obj,
                defaults={
                    'flooring': features_data.get('flooring'),
                    'foundation_details': features_data.get('foundationDetails'),
                    'accessibility_features': features_data.get('accessibilityFeatures'),
                    'garage_spaces': features_data.get('garageSpaces'),
                    'parking_spaces': features_data.get('parkingSpaces'),
                    'view_type': features_data.get('viewType'),
                    'water_view': features_data.get('waterViewYN', False),
                    'heating': features_data.get('heating'),
                    'cooling': features_data.get('cooling'),
                    'construction_materials': features_data.get('constructionMaterials'),
                    'roof_type': features_data.get('roofType'),
                    'lot_size': features_data.get('lotSize'),
                    'hoa_fee': features_data.get('hoaFee'),
                }
            )

            # Handle Agents
            for agent_data in property_data.get('contact_recipients', []):
                agent_defaults = {
                    'display_name': agent_data['display_name'],
                    'review_count': agent_data['review_count'],
                    'rating_average': agent_data['rating_average'],
                    'phone_number': f"{agent_data['phone']['areacode']}-{agent_data['phone']['prefix']}-{agent_data['phone']['number']}",
                    'image_url': agent_data['image_url'],
                    'badge_type': agent_data['badge_type'],
                }
                agent_obj, agent_created = Agent.objects.update_or_create(
                    display_name=agent_data['display_name'], defaults=agent_defaults
                )
                PropertyAgent.objects.update_or_create(property=property_obj, agent=agent_obj)

            # Handle Schools
            for school_data in property_data.get('schools', []):
                school_defaults = {
                    'name': school_data['name'],
                    'rating': school_data['rating'],
                    'students_per_teacher': school_data['studentsPerTeacher'],
                    'size': school_data['size'],
                    'level': school_data['level'],
                    'grades': school_data['grades'],
                    'type': school_data['type'],
                    'distance': school_data['distance'],
                }
                school_obj, school_created = School.objects.update_or_create(
                    name=school_data['name'], defaults=school_defaults
                )
                PropertySchool.objects.update_or_create(property=property_obj, school=school_obj)

            # Handle PriceHistory
            for price_history_data in property_data.get('priceHistory', []):
                PriceHistory.objects.update_or_create(
                    property=property_obj,
                    date=parse_datetime_safe(price_history_data['date']),
                    defaults={
                        'event': price_history_data['event'],
                        'price': price_history_data['price'],
                        'price_per_square_foot': price_history_data.get('pricePerSquareFoot'),
                    }
                )

            # Handle NearbyHomes
            for nearby_home_data in property_data.get('nearbyHomes', []):
                NearbyHomes.objects.update_or_create(
                    property=property_obj,
                    zpid=nearby_home_data['zpid'],
                    defaults={
                        'longitude': nearby_home_data['longitude'],
                        'latitude': nearby_home_data['latitude'],
                        'price': nearby_home_data['price'],
                        'home_type': nearby_home_data['homeType'],
                        'home_status': nearby_home_data['homeStatus'],
                    }
                )

In [9]:
property_dictionary_list = propertylist.property_dictionary_list
save_properties_and_related_data(property_dictionary_list)

ValueError: Field 'time_on_zillow' expected a number but got '129 days'.